In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 999
pd.options.display.max_columns = 999
import os
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
#from wordcloud import WordCloud
import pandas_profiling
import missingno as msno
import datetime
import re

## This file shows how we validate hypothesis 3:
### Time for positive/negative/anger/anxiety/sad tweets drop to half peak volume is longer in 2013 than 2012.

feel free to contact me(yl3963@columbia.edu) if you have any questions


## prepare dataset

In [102]:
twitter1 = pd.read_csv("tweet original data/Trayvon_Tweets2012.csv",encoding = "latin1",index_col = 0)
twitter2 = pd.read_csv("tweet original data/TrayvonTweets2013.csv",encoding = "latin1", index_col = 0)
twitter = pd.concat([twitter1,twitter2],ignore_index=True)

In [103]:
twitter=twitter[['date_posted_gmt','message']]
twitter = twitter.loc[twitter['message'].isnull() == False,['date_posted_gmt','message']]


In [104]:
twitter.head()

,date_posted_gmt,message
0,40960.950984,Buildings instructions for Martin LattaÃ¢â¬â...
1,40960.979954,Ã¢â¬Å@DrKillJoyXD: I was never a fan of Mart...
2,40960.984468,#MakesMeSMH When I See Black Violence. Didn't ...
3,40960.984664,In flight movie was The Big Year. A bird watch...
4,40961.121262,Trayvon-conrad Webster interview by Aden Bashr...


In [324]:
twitter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2094583 entries, 0 to 2097149
Data columns (total 8 columns):
date_posted_gmt    object
message            object
date               datetime64[ns]
year               int64
month              int64
day                int64
week               int64
days_from_event    int64
dtypes: datetime64[ns](1), int64(5), object(2)
memory usage: 143.8+ MB


In [105]:
def date(dates):
    delta=datetime.timedelta(days=dates)
    today=datetime.datetime.strptime('1899-12-30 00:00:00','%Y-%m-%d %H:%M:%S')+delta
    return datetime.datetime.strftime(today,'%Y-%m-%d %H:%M:%S')


In [106]:
twitter['date_posted_gmt'] = twitter['date_posted_gmt'].apply(lambda x: date(x))


In [107]:
twitter.head()

,date_posted_gmt,message
0,2012-02-21 22:49:25,Buildings instructions for Martin LattaÃ¢â¬â...
1,2012-02-21 23:31:08,Ã¢â¬Å@DrKillJoyXD: I was never a fan of Mart...
2,2012-02-21 23:37:38,#MakesMeSMH When I See Black Violence. Didn't ...
3,2012-02-21 23:37:55,In flight movie was The Big Year. A bird watch...
4,2012-02-22 02:54:37,Trayvon-conrad Webster interview by Aden Bashr...


####  function add_ymdw: add year month day  week columns for original dataframe 


In [108]:
def add_ymdw(df):
    df['date']=pd.to_datetime(df['date_posted_gmt'])

    df['year']=df['date'].apply(lambda x: x.year)

    df['month']=df['date'].apply(lambda x: x.month)

    df['day']=df['date'].apply(lambda x: x.day)
    
    def getweek(a,b,c):
        return datetime.date(a,b,c).isocalendar()[1]
    df['week'] = df.apply(lambda x: getweek(x['year'], x['month'],x['day']), axis=1)

    return df

In [109]:
data_msg_time=add_ymdw(twitter)
data_msg_time.head()

,date_posted_gmt,message,date,year,month,day,week
0,2012-02-21 22:49:25,Buildings instructions for Martin LattaÃ¢â¬â...,2012-02-21 22:49:25,2012,2,21,8
1,2012-02-21 23:31:08,Ã¢â¬Å@DrKillJoyXD: I was never a fan of Mart...,2012-02-21 23:31:08,2012,2,21,8
2,2012-02-21 23:37:38,#MakesMeSMH When I See Black Violence. Didn't ...,2012-02-21 23:37:38,2012,2,21,8
3,2012-02-21 23:37:55,In flight movie was The Big Year. A bird watch...,2012-02-21 23:37:55,2012,2,21,8
4,2012-02-22 02:54:37,Trayvon-conrad Webster interview by Aden Bashr...,2012-02-22 02:54:37,2012,2,22,8


### split dataset according to their year
you can get 3 dataframes finally

In [325]:
def split_year(df):
    df2012=df[df['year']==2012]
    df2013=df[df['year']==2013]
    df2014=df[df['year']==2014]
    return df2012,df2013,df2014

In [326]:
d2012,d2013,d2014=split_year(data_msg_time)
d2013.head()

,date_posted_gmt,message,date,year,month,day,week,days_from_event
96948,2013-01-01 03:47:00,RIP to Trayvon,2013-01-01 03:47:00,2013,1,1,1,-193
96949,2013-01-01 03:59:52,Reflecting On 2012: Sybrina Fulton Speaks On H...,2013-01-01 03:59:52,2013,1,1,1,-193
96950,2013-01-01 04:40:15,"Bringing in 2013, we won't forget Trayvon Mart...",2013-01-01 04:40:15,2013,1,1,1,-193
96951,2013-01-01 04:42:07,I liked a @YouTube video from @HoodNews http:/...,2013-01-01 04:42:07,2013,1,1,1,-193
96952,2013-01-01 04:45:19,Got My Hoodie On To Cuh !Ã¢â¬Å@CuzTATTED Got...,2013-01-01 04:45:19,2013,1,1,1,-193


In [327]:
event_date = '20120226'
d2012['days_from_event'] = d2012.apply(lambda x: (x['date'] - pd.to_datetime(event_date)).days, axis=1)
d2012.head()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,date_posted_gmt,message,date,year,month,day,week,days_from_event
0,2012-02-21 22:49:25,Buildings instructions for Martin LattaÃ¢â¬â...,2012-02-21 22:49:25,2012,2,21,8,-5
1,2012-02-21 23:31:08,Ã¢â¬Å@DrKillJoyXD: I was never a fan of Mart...,2012-02-21 23:31:08,2012,2,21,8,-5
2,2012-02-21 23:37:38,#MakesMeSMH When I See Black Violence. Didn't ...,2012-02-21 23:37:38,2012,2,21,8,-5
3,2012-02-21 23:37:55,In flight movie was The Big Year. A bird watch...,2012-02-21 23:37:55,2012,2,21,8,-5
4,2012-02-22 02:54:37,Trayvon-conrad Webster interview by Aden Bashr...,2012-02-22 02:54:37,2012,2,22,8,-4


In [328]:
event_date = '20130713'
d2013['days_from_event'] = d2013.apply(lambda x: (x['date'] - pd.to_datetime(event_date)).days, axis=1)


/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,date_posted_gmt,message,date,year,month,day,week,days_from_event
0,2012-02-21 22:49:25,Buildings instructions for Martin LattaÃ¢â¬â...,2012-02-21 22:49:25,2012,2,21,8,-5
1,2012-02-21 23:31:08,Ã¢â¬Å@DrKillJoyXD: I was never a fan of Mart...,2012-02-21 23:31:08,2012,2,21,8,-5
2,2012-02-21 23:37:38,#MakesMeSMH When I See Black Violence. Didn't ...,2012-02-21 23:37:38,2012,2,21,8,-5
3,2012-02-21 23:37:55,In flight movie was The Big Year. A bird watch...,2012-02-21 23:37:55,2012,2,21,8,-5
4,2012-02-22 02:54:37,Trayvon-conrad Webster interview by Aden Bashr...,2012-02-22 02:54:37,2012,2,22,8,-4


In [329]:
d2013['message'] = d2013['message'].str.lower()
d2012['message'] = d2012['message'].str.lower()

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [330]:
d2012.head()

,date_posted_gmt,message,date,year,month,day,week,days_from_event
0,2012-02-21 22:49:25,buildings instructions for martin lattaã¢â¬â...,2012-02-21 22:49:25,2012,2,21,8,-5
1,2012-02-21 23:31:08,ã¢â¬å@drkilljoyxd: i was never a fan of mart...,2012-02-21 23:31:08,2012,2,21,8,-5
2,2012-02-21 23:37:38,#makesmesmh when i see black violence. didn't ...,2012-02-21 23:37:38,2012,2,21,8,-5
3,2012-02-21 23:37:55,in flight movie was the big year. a bird watch...,2012-02-21 23:37:55,2012,2,21,8,-5
4,2012-02-22 02:54:37,trayvon-conrad webster interview by aden bashr...,2012-02-22 02:54:37,2012,2,22,8,-4


In [456]:
d2012.to_csv('d2012.csv',encoding='utf-8')

In [457]:
d2013.to_csv('d2013.csv',encoding='utf-8')

In [331]:
d2013.head()

,date_posted_gmt,message,date,year,month,day,week,days_from_event
96948,2013-01-01 03:47:00,rip to trayvon,2013-01-01 03:47:00,2013,1,1,1,-193
96949,2013-01-01 03:59:52,reflecting on 2012: sybrina fulton speaks on h...,2013-01-01 03:59:52,2013,1,1,1,-193
96950,2013-01-01 04:40:15,"bringing in 2013, we won't forget trayvon mart...",2013-01-01 04:40:15,2013,1,1,1,-193
96951,2013-01-01 04:42:07,i liked a @youtube video from @hoodnews http:/...,2013-01-01 04:42:07,2013,1,1,1,-193
96952,2013-01-01 04:45:19,got my hoodie on to cuh !ã¢â¬å@cuztatted got...,2013-01-01 04:45:19,2013,1,1,1,-193


In [115]:
file = open("LIWC Dictionary/LIWC_dic.txt","r")

LIWC = file.readlines()
LIWC_dict = {}
for line in LIWC:
    line = line.split(',')
    category = line[0]
    words = line[1:]
    words[0] = words[0][1:]
    words[-1] = words[-1][:-2]
    LIWC_dict[category] = words

In [116]:
posemo = LIWC_dict['Posemo']
for i in range(len(posemo)):
    if posemo[i][-1] == '*':
        posemo[i] = r'\b' + posemo[i][:-1]
    else:
        posemo[i] = r'\b' + posemo[i] + r'\b'
posemo[0] = '\(:'
posemo[5] = ':\)'
reg_posemo = "|".join(posemo)
reg_posemo = re.compile(reg_posemo)

negemo = LIWC_dict['Negemo']
for i in range(len(negemo)):
    if negemo[i][-1] == '*':
        negemo[i] = r'\b' + negemo[i][:-1]
    else:
        negemo[i] = r'\b' + negemo[i] + r'\b'
negemo[0] = '\):'
negemo[5] = ':\('
reg_negemo = "|".join(negemo)
reg_negemo = re.compile(reg_negemo)

anx = LIWC_dict['Anx']
for i in range(len(anx)):
    if anx[i][-1] == '*':
        anx[i] = r'\b' + anx[i][:-1]
    else:
        anx[i] = r'\b' + anx[i] + r'\b'
reg_anx = "|".join(anx)
reg_anx = re.compile(reg_anx)

anger = LIWC_dict['Anger']
for i in range(len(anger)):
    if anger[i][-1] == '*':
        anger[i] = r'\b' + anger[i][:-1]
    else:
        anger[i] = r'\b' + anger[i] + r'\b'
reg_anger = "|".join(anger)
reg_anger = re.compile(reg_anger)

sad = LIWC_dict['Sad']
for i in range(len(sad)):
    if sad[i][-1] == '*':
        sad[i] = r'\b' + sad[i][:-1]
    else:
        sad[i] = r'\b' + sad[i] + r'\b'
reg_sad = "|".join(sad)
reg_sad = re.compile(reg_sad)

stress = LIWC_dict['Stress']
for i in range(len(stress)):
    if stress[i][-1] == '*':
        stress[i] = r'\b' + stress[i][:-1]
    else:
        stress[i] = r'\b' + stress[i] + r'\b'
reg_stress = "|".join(stress)
reg_stress = re.compile(reg_stress)

In [117]:
def check_category(s,category,dic):
    '''
    s type: string
    category type: list of string 
    '''
    if category == 'Posemo':
        if reg_posemo.search(s):
            return category
    elif category == 'Negemo':
        if reg_negemo.search(s):
            return category
    elif category == 'Anx':
        if reg_anx.search(s):
            return category
    elif category == 'Anger':
        if reg_anger.search(s):
            return category
    elif category == 'Sad':
        if reg_sad.search(s):
            return category
    elif category == 'Stress':
        if reg_stress.search(s):
            return category
    else:
        print("only valid for six category")
    return 'NA'

In [118]:
check_category('stressed','sad',LIWC_dict)

only valid for six category


'NA'

### add_tag_col
- df:  dataframe 
- cate: emotion category
- dic: the dictionary stores all words belong to the category
e.g. here tag='anx' means the message of the row contains anxiety words

In [332]:
d2012_1=d2012.sample(n=100)

In [336]:
d2013_1=d2013.sample(n=100)

In [334]:
def add_tag_col(df,cate,dic):
    for category in cate:
        df[category]=df.apply(lambda x: check_category(x['message'],category,dic),axis=1)
    return df

In [335]:
emotions = ['Sad','Anx','Anger','Negemo','Posemo','Stress']

d2012_1=add_tag_col(d2012_1,emotions,LIWC_dict)  
d2012_1.head()

,date_posted_gmt,message,date,year,month,day,week,days_from_event,Sad,Anx,Anger,Negemo,Posemo,Stress
247383,2012-03-11 22:44:33,just curious: have any civil rights leaders ma...,2012-03-11 22:44:33,2012,3,11,10,14,NA,NA,NA,NA,NA,NA
787129,2012-03-27 01:45:08,"#baowwww ...... george w. zimmerman, 159 edgew...",2012-03-27 01:45:08,2012,3,27,13,30,NA,NA,NA,NA,NA,NA
281018,2012-03-24 18:59:38,trayvon martin case: rev. jesse jackson says i...,2012-03-24 18:59:38,2012,3,24,12,27,NA,NA,Anger,Negemo,NA,NA
330799,2012-04-12 00:50:32,trayvon case is setting up to be rodney king a...,2012-04-12 00:50:32,2012,4,12,15,46,NA,NA,NA,NA,Posemo,NA
917267,2012-03-28 13:05:54,so heartbreaking. family photos: the final day...,2012-03-28 13:05:54,2012,3,28,13,31,Sad,NA,NA,Negemo,NA,NA


In [337]:
d2013_1=add_tag_col(d2013_1,emotions,LIWC_dict)  
d2013_1.head()

,date_posted_gmt,message,date,year,month,day,week,days_from_event,Sad,Anx,Anger,Negemo,Posemo,Stress
2033611,2013-07-14 00:11:28,the trayvon martin case should teach one lesso...,2013-07-14 00:11:28,2013,7,14,28,1,NA,NA,Anger,Negemo,Posemo,NA
2022717,2013-07-12 09:43:05,news... defense to make final arguments in geo...,2013-07-12 09:43:05,2013,7,12,28,-1,NA,NA,Anger,Negemo,NA,NA
1788610,2013-07-16 03:52:20,juror says both of them could have walked away...,2013-07-16 03:52:20,2013,7,16,29,3,NA,NA,NA,NA,NA,NA
1538854,2013-07-20 21:21:02,"per the ap, jay z and beyonce were at the tray...",2013-07-20 21:21:02,2013,7,20,29,7,NA,NA,NA,NA,NA,NA
1212863,2013-07-14 04:52:09,everybody wants to get problack because of the...,2013-07-14 04:52:09,2013,7,14,28,1,NA,NA,NA,NA,Posemo,NA


In [259]:
def form_table(df,cate):
    df_cate_week=df[df[cate]==cate].groupby('week')['message'].count()
    all_res=df.groupby('week')['message'].count()
    catelist=[]
    for week,val in df_cate_week.items():
        catelist.append((week,val))
#     catelist
    alltweetlist=[]
    for week,val in all_res.items():
        alltweetlist.append((week,val))
    # alltweetlist
    final0=[]
    for i in range(0,53):
        for week1,catenum in catelist:
            if week1!=i:
                final0.append((i,0))
                continue
            for week2,tweetnum in alltweetlist:
                if week1==week2:
                    final0.append((week1,100*catenum/tweetnum))
                    continue
    final0=list(set(final0))
    final0.sort()

    final1={}
    for a,b in final0:
        if a not in final1:
            final1[a]=b
        else:
            final1[a]+=b
    cate_tb=pd.DataFrame(list(final1.items()), columns=['week', cate])
    return cate_tb

In [310]:
sad_2012_1=form_table(d2012_1,'Negemo')
sad_2012_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53 entries, 0 to 52
Data columns (total 2 columns):
week      53 non-null int64
Negemo    53 non-null float64
dtypes: float64(1), int64(1)
memory usage: 928.0 bytes


In [311]:
sad_2012_1

,week,Negemo
0,0,0.000000
1,1,0.000000
2,2,0.000000
3,3,0.000000
4,4,0.000000
5,5,0.000000
6,6,0.000000
7,7,0.000000
8,8,0.000000
9,9,9.090909


In [340]:
def find_half_peak_time_diff(df,cate):
    
    def find_peak_time(df,cate):
        return int(df[cate].values.argmax())
    
    def find_peak_value(df,cate):
        return float(df.iloc[find_peak_time(df,cate)][cate])
    
    peaktime=find_peak_time(df,cate)
#     print(peaktime)
    
    peakval=find_peak_value(df,cate)
#     print(peakval)
    half=peakval/2
    for week in range(peaktime+1,53):
        if df.iloc[week][cate]<=half:
            for week2 in range(week+1,53):
                if df.iloc[week2][cate]<=half:
                     return week-peaktime
    return 0
        
find_half_peak_time_diff(sad_2012_1,'Negemo')

1

In [366]:
def bootstrap_diff(df,cate,epoch):
    emotion=[]
    for epoch in range(epoch):
        df_sample=df.sample(n=20000)
        emotions = [cate]
        df_sample=add_tag_col(df_sample,emotions,LIWC_dict)  
        df_sample=form_table(df_sample,cate)
        time_diff_2012=find_half_peak_time_diff(df_sample,cate)
        emotion.append(time_diff_2012)
        print('epoch',epoch,'diff is:',time_diff_2012)
    return emotion,sum(emotion)/len(emotion)

## discover negemo

In [372]:
bootstrap_diff(d2012,'Negemo',30)

epoch 0 diff is: 1
epoch 1 diff is: 2
epoch 2 diff is: 2
epoch 3 diff is: 1
epoch 4 diff is: 1
epoch 5 diff is: 2
epoch 6 diff is: 3
epoch 7 diff is: 1
epoch 8 diff is: 1
epoch 9 diff is: 2
epoch 10 diff is: 1
epoch 11 diff is: 1
epoch 12 diff is: 2
epoch 13 diff is: 2
epoch 14 diff is: 1
epoch 15 diff is: 1
epoch 16 diff is: 1
epoch 17 diff is: 2
epoch 18 diff is: 1
epoch 19 diff is: 1
epoch 20 diff is: 1
epoch 21 diff is: 1
epoch 22 diff is: 1
epoch 23 diff is: 1
epoch 24 diff is: 1
epoch 25 diff is: 1
epoch 26 diff is: 1
epoch 27 diff is: 1
epoch 28 diff is: 2
epoch 29 diff is: 1


([1,
  2,
  2,
  1,
  1,
  2,
  3,
  1,
  1,
  2,
  1,
  1,
  2,
  2,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1],
 1.3333333333333333)

## discover Sad

In [368]:
bootstrap_diff(d2012,'Sad',30)

epoch 0 diff is: 1
epoch 1 diff is: 1
epoch 2 diff is: 1
epoch 3 diff is: 1
epoch 4 diff is: 1
epoch 5 diff is: 2
epoch 6 diff is: 1
epoch 7 diff is: 1
epoch 8 diff is: 1
epoch 9 diff is: 1
epoch 10 diff is: 0
epoch 11 diff is: 1
epoch 12 diff is: 0
epoch 13 diff is: 1
epoch 14 diff is: 1
epoch 15 diff is: 1
epoch 16 diff is: 1
epoch 17 diff is: 1
epoch 18 diff is: 1
epoch 19 diff is: 1
epoch 20 diff is: 1
epoch 21 diff is: 2
epoch 22 diff is: 1
epoch 23 diff is: 1
epoch 24 diff is: 1
epoch 25 diff is: 1
epoch 26 diff is: 1
epoch 27 diff is: 1
epoch 28 diff is: 3
epoch 29 diff is: 1


([1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  0,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  1],
 1.0666666666666667)

# discover Anxiety

In [369]:
bootstrap_diff(d2012,'Anx',30)

epoch 0 diff is: 2
epoch 1 diff is: 1
epoch 2 diff is: 1
epoch 3 diff is: 1
epoch 4 diff is: 1
epoch 5 diff is: 1
epoch 6 diff is: 1
epoch 7 diff is: 1
epoch 8 diff is: 1
epoch 9 diff is: 1
epoch 10 diff is: 2
epoch 11 diff is: 1
epoch 12 diff is: 1
epoch 13 diff is: 2
epoch 14 diff is: 3
epoch 15 diff is: 1
epoch 16 diff is: 1
epoch 17 diff is: 1
epoch 18 diff is: 1
epoch 19 diff is: 2
epoch 20 diff is: 1
epoch 21 diff is: 1
epoch 22 diff is: 1
epoch 23 diff is: 1
epoch 24 diff is: 1
epoch 25 diff is: 1
epoch 26 diff is: 1
epoch 27 diff is: 2
epoch 28 diff is: 1
epoch 29 diff is: 1


([2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  2,
  3,
  1,
  1,
  1,
  1,
  2,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  1],
 1.2333333333333334)

## discover anger

In [370]:
bootstrap_diff(d2012,'Anger',30)

epoch 0 diff is: 1
epoch 1 diff is: 1
epoch 2 diff is: 1
epoch 3 diff is: 1
epoch 4 diff is: 1
epoch 5 diff is: 1
epoch 6 diff is: 1
epoch 7 diff is: 1
epoch 8 diff is: 1
epoch 9 diff is: 1
epoch 10 diff is: 1
epoch 11 diff is: 1
epoch 12 diff is: 1
epoch 13 diff is: 1
epoch 14 diff is: 1
epoch 15 diff is: 1
epoch 16 diff is: 1
epoch 17 diff is: 1
epoch 18 diff is: 1
epoch 19 diff is: 1
epoch 20 diff is: 1
epoch 21 diff is: 1
epoch 22 diff is: 1
epoch 23 diff is: 3
epoch 24 diff is: 1
epoch 25 diff is: 1
epoch 26 diff is: 1
epoch 27 diff is: 1
epoch 28 diff is: 1
epoch 29 diff is: 1


([1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  1,
  1,
  1,
  1,
  1,
  1],
 1.0666666666666667)

## discover stress

In [371]:
bootstrap_diff(d2012,'Stress',30)

epoch 0 diff is: 1
epoch 1 diff is: 1
epoch 2 diff is: 1
epoch 3 diff is: 1
epoch 4 diff is: 1
epoch 5 diff is: 1
epoch 6 diff is: 1
epoch 7 diff is: 1
epoch 8 diff is: 1
epoch 9 diff is: 1
epoch 10 diff is: 1
epoch 11 diff is: 1
epoch 12 diff is: 1
epoch 13 diff is: 1
epoch 14 diff is: 1
epoch 15 diff is: 1
epoch 16 diff is: 1
epoch 17 diff is: 1
epoch 18 diff is: 2
epoch 19 diff is: 1
epoch 20 diff is: 0
epoch 21 diff is: 1
epoch 22 diff is: 1
epoch 23 diff is: 1
epoch 24 diff is: 1
epoch 25 diff is: 1
epoch 26 diff is: 1
epoch 27 diff is: 1
epoch 28 diff is: 1
epoch 29 diff is: 1


([1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  2,
  1,
  0,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 1.0)

## discover  posemo

In [373]:
bootstrap_diff(d2012,'Posemo',30)

epoch 0 diff is: 2
epoch 1 diff is: 3
epoch 2 diff is: 1
epoch 3 diff is: 3
epoch 4 diff is: 5
epoch 5 diff is: 7
epoch 6 diff is: 1
epoch 7 diff is: 3
epoch 8 diff is: 2
epoch 9 diff is: 5
epoch 10 diff is: 3
epoch 11 diff is: 1
epoch 12 diff is: 3
epoch 13 diff is: 3
epoch 14 diff is: 6
epoch 15 diff is: 1
epoch 16 diff is: 3
epoch 17 diff is: 1
epoch 18 diff is: 6
epoch 19 diff is: 7
epoch 20 diff is: 1
epoch 21 diff is: 9
epoch 22 diff is: 5
epoch 23 diff is: 5
epoch 24 diff is: 3
epoch 25 diff is: 2
epoch 26 diff is: 3
epoch 27 diff is: 2
epoch 28 diff is: 5
epoch 29 diff is: 3


([2,
  3,
  1,
  3,
  5,
  7,
  1,
  3,
  2,
  5,
  3,
  1,
  3,
  3,
  6,
  1,
  3,
  1,
  6,
  7,
  1,
  9,
  5,
  5,
  3,
  2,
  3,
  2,
  5,
  3],
 3.466666666666667)

## due to missing patterns in 2013 dataset, we decide to only consider records after week 28

In [ ]:
def find_half_peak_time_diff(df,cate):
    
    def find_peak_time(df,cate):
        return int(df[cate].values.argmax())
    
    def find_peak_value(df,cate):
        return float(df.iloc[find_peak_time(df,cate)][cate])
    
    peaktime=find_peak_time(df,cate)
#     print(peaktime)
    
    peakval=find_peak_value(df,cate)
#     print(peakval)
    half=peakval/2
    for week in range(peaktime+1,53):
        if df.iloc[week][cate]<=half:
            for week2 in range(week+1,53):
                if df.iloc[week2][cate]<=half:
                     return week-peaktime
    return 0
        
find_half_peak_time_diff(sad_2012_1,'Negemo')

In [418]:
df=d2013.sample(30000)
df_sample=df.sample(n=20000)
cate='Sad'
emotions = [cate]
df_sample=add_tag_col(df_sample,emotions,LIWC_dict)  
df_sample=form_table(df_sample,cate)

In [419]:
df_sample

,week,Sad
0,0,0.000000
1,1,4.285714
2,2,4.347826
3,3,7.894737
4,4,1.973684
5,5,8.196721
6,6,2.425876
7,7,1.694915
8,8,0.000000
9,9,2.325581


In [420]:
df_sample=df_sample.iloc[28:,:]# compute from week 28
df_sample

,week,Sad
28,28,6.264775
29,29,6.828852
30,30,5.184260
31,31,3.018868
32,32,3.643725
33,33,1.578947
34,34,3.358209
35,35,2.777778
36,36,2.339181
37,37,4.054054


In [440]:

def find_peak_time_2013(df,cate):
        return int(28+df[cate].values.argmax())
    
peaktime=find_peak_time(df_sample,cate)
peaktime

49

In [441]:
df_sample.iloc[peaktime-28]

week    49.000000
Sad      7.317073
Name: 49, dtype: float64

In [442]:
def find_peak_value_2013(df,cate):
        return float(df.iloc[peaktime-28][cate])
find_peak_value(df_sample,cate)    

7.317073170731708

In [445]:
def find_half_peak_time_diff_2013(df,cate):
    
    def find_peak_time_2013(df,cate):
        return int(28+df[cate].values.argmax())
    
    peaktime=find_peak_time(df,cate)-28
    
    def find_peak_value_2013(df,cate):
        return float(df.iloc[find_peak_time(df,cate)-28][cate])
    
    peakval=find_peak_value(df,cate)
    half=peakval/2
    
    for week in range(peaktime+1,53-28):
        if df.iloc[week][cate]<=half:
            for week2 in range(week+1,53-28):
                if df.iloc[week2][cate]<=half:
                     return week-peaktime
    return 0

In [446]:
find_half_peak_time_diff_2013(df_sample,cate)

1

In [447]:
def bootstrap_diff_2013(df,cate,epoch):
    emotion=[]
    
    for epoch in range(epoch):
        df_sample=df.sample(n=20000)
        emotions = [cate]
        df_sample=add_tag_col(df_sample,emotions,LIWC_dict)  
        df_sample=form_table(df_sample,cate)
        df_sample=df_sample.iloc[28:,:]# compute from week 28
        time_diff_2013=find_half_peak_time_diff_2013(df_sample,cate)
        emotion.append(time_diff_2013)
        print('epoch',epoch,'diff is:',time_diff_2013)
    return emotion,sum(emotion)/len(emotion)

## sad in 2013

In [448]:
bootstrap_diff_2013(d2013,'Sad',30)

epoch 0 diff is: 2
epoch 1 diff is: 1
epoch 2 diff is: 1
epoch 3 diff is: 0
epoch 4 diff is: 0
epoch 5 diff is: 1
epoch 6 diff is: 1
epoch 7 diff is: 20
epoch 8 diff is: 2
epoch 9 diff is: 7
epoch 10 diff is: 3
epoch 11 diff is: 2
epoch 12 diff is: 1
epoch 13 diff is: 0
epoch 14 diff is: 5
epoch 15 diff is: 2
epoch 16 diff is: 2
epoch 17 diff is: 0
epoch 18 diff is: 1
epoch 19 diff is: 0
epoch 20 diff is: 11
epoch 21 diff is: 1
epoch 22 diff is: 2
epoch 23 diff is: 0
epoch 24 diff is: 2
epoch 25 diff is: 4
epoch 26 diff is: 0
epoch 27 diff is: 0
epoch 28 diff is: 1
epoch 29 diff is: 2


([2,
  1,
  1,
  0,
  0,
  1,
  1,
  20,
  2,
  7,
  3,
  2,
  1,
  0,
  5,
  2,
  2,
  0,
  1,
  0,
  11,
  1,
  2,
  0,
  2,
  4,
  0,
  0,
  1,
  2],
 2.466666666666667)

## stress in 2013

In [449]:
bootstrap_diff_2013(d2013,'Stress',30)

epoch 0 diff is: 0
epoch 1 diff is: 0
epoch 2 diff is: 0
epoch 3 diff is: 4
epoch 4 diff is: 5
epoch 5 diff is: 5
epoch 6 diff is: 7
epoch 7 diff is: 0
epoch 8 diff is: 1
epoch 9 diff is: 2
epoch 10 diff is: 2
epoch 11 diff is: 4
epoch 12 diff is: 0
epoch 13 diff is: 2
epoch 14 diff is: 7
epoch 15 diff is: 0
epoch 16 diff is: 2
epoch 17 diff is: 1
epoch 18 diff is: 1
epoch 19 diff is: 1
epoch 20 diff is: 0
epoch 21 diff is: 1
epoch 22 diff is: 3
epoch 23 diff is: 6
epoch 24 diff is: 3
epoch 25 diff is: 3
epoch 26 diff is: 2
epoch 27 diff is: 3
epoch 28 diff is: 2
epoch 29 diff is: 2


([0,
  0,
  0,
  4,
  5,
  5,
  7,
  0,
  1,
  2,
  2,
  4,
  0,
  2,
  7,
  0,
  2,
  1,
  1,
  1,
  0,
  1,
  3,
  6,
  3,
  3,
  2,
  3,
  2,
  2],
 2.3)

### anger in 2013

In [450]:
bootstrap_diff_2013(d2013,'Anger',30)

epoch 0 diff is: 0
epoch 1 diff is: 0
epoch 2 diff is: 0
epoch 3 diff is: 0
epoch 4 diff is: 0
epoch 5 diff is: 0
epoch 6 diff is: 0
epoch 7 diff is: 0
epoch 8 diff is: 0
epoch 9 diff is: 0
epoch 10 diff is: 0
epoch 11 diff is: 0
epoch 12 diff is: 0
epoch 13 diff is: 0
epoch 14 diff is: 0
epoch 15 diff is: 0
epoch 16 diff is: 0
epoch 17 diff is: 0
epoch 18 diff is: 0
epoch 19 diff is: 0
epoch 20 diff is: 0
epoch 21 diff is: 0
epoch 22 diff is: 0
epoch 23 diff is: 0
epoch 24 diff is: 0
epoch 25 diff is: 0
epoch 26 diff is: 0
epoch 27 diff is: 0
epoch 28 diff is: 0
epoch 29 diff is: 0


([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 0.0)

### Anxiety in 2013

In [451]:
bootstrap_diff_2013(d2013,'Anx',30)

epoch 0 diff is: 2
epoch 1 diff is: 2
epoch 2 diff is: 0
epoch 3 diff is: 1
epoch 4 diff is: 2
epoch 5 diff is: 10
epoch 6 diff is: 0
epoch 7 diff is: 4
epoch 8 diff is: 3
epoch 9 diff is: 7
epoch 10 diff is: 0
epoch 11 diff is: 4
epoch 12 diff is: 0
epoch 13 diff is: 0
epoch 14 diff is: 0
epoch 15 diff is: 2
epoch 16 diff is: 0
epoch 17 diff is: 0
epoch 18 diff is: 7
epoch 19 diff is: 0
epoch 20 diff is: 3
epoch 21 diff is: 3
epoch 22 diff is: 12
epoch 23 diff is: 4
epoch 24 diff is: 3
epoch 25 diff is: 1
epoch 26 diff is: 2
epoch 27 diff is: 0
epoch 28 diff is: 0
epoch 29 diff is: 1


([2,
  2,
  0,
  1,
  2,
  10,
  0,
  4,
  3,
  7,
  0,
  4,
  0,
  0,
  0,
  2,
  0,
  0,
  7,
  0,
  3,
  3,
  12,
  4,
  3,
  1,
  2,
  0,
  0,
  1],
 2.433333333333333)

### posemo in 2013

In [453]:
bootstrap_diff_2013(d2013,'Posemo',30)

epoch 0 diff is: 0
epoch 1 diff is: 0
epoch 2 diff is: 0
epoch 3 diff is: 0
epoch 4 diff is: 0
epoch 5 diff is: 0
epoch 6 diff is: 0
epoch 7 diff is: 0
epoch 8 diff is: 0
epoch 9 diff is: 0
epoch 10 diff is: 0
epoch 11 diff is: 0
epoch 12 diff is: 0
epoch 13 diff is: 0
epoch 14 diff is: 0
epoch 15 diff is: 0
epoch 16 diff is: 0
epoch 17 diff is: 0
epoch 18 diff is: 0
epoch 19 diff is: 0
epoch 20 diff is: 0
epoch 21 diff is: 0
epoch 22 diff is: 0
epoch 23 diff is: 0
epoch 24 diff is: 0
epoch 25 diff is: 0
epoch 26 diff is: 0
epoch 27 diff is: 0
epoch 28 diff is: 0
epoch 29 diff is: 0


([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 0.0)

### negemo in 2013

In [452]:
bootstrap_diff_2013(d2013,'Negemo',30)

epoch 0 diff is: 0
epoch 1 diff is: 0
epoch 2 diff is: 0
epoch 3 diff is: 0
epoch 4 diff is: 0
epoch 5 diff is: 0
epoch 6 diff is: 0
epoch 7 diff is: 0
epoch 8 diff is: 0
epoch 9 diff is: 0
epoch 10 diff is: 0
epoch 11 diff is: 0
epoch 12 diff is: 0
epoch 13 diff is: 0
epoch 14 diff is: 0
epoch 15 diff is: 0
epoch 16 diff is: 0
epoch 17 diff is: 0
epoch 18 diff is: 0
epoch 19 diff is: 0
epoch 20 diff is: 0
epoch 21 diff is: 0
epoch 22 diff is: 0
epoch 23 diff is: 0
epoch 24 diff is: 0
epoch 25 diff is: 0
epoch 26 diff is: 0
epoch 27 diff is: 0
epoch 28 diff is: 0
epoch 29 diff is: 0


([0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 0.0)